In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn
try: from sklearn.model_selection import train_test_split
except: from sklearn.cross_validation import train_test_split

from sklearn import datasets
from djinn import djinn
print(sklearn.__version__)

Instructions for updating:
non-resource variables are not supported in the long term
1.0.2


In [ ]:
#Load the data, split into training/testing groups
d=datasets.load_iris()
X=d.data
Y=d.target

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X, Y, test_size=0.2, random_state=1) 

In [ ]:
print("djinn example")    
modelname="class_djinn_test"   # name the model
ntrees=1                 # number of trees = number of neural nets in ensemble
maxdepth=4               # max depth of tree -- optimize this for each data set
dropout_keep=1.0         # dropout typically set to 1 for non-Bayesian models

#initialize the model
model=djinn.DJINN_Classifier(ntrees,maxdepth,dropout_keep)

djinn example


In [ ]:
# find optimal settings: this function returns dict with hyper-parameters
# each djinn function accepts random seeds for reproducible behavior
optimal=model.get_hyperparameters(x_train, y_train, random_state=1)
batchsize=optimal['batch_size']
learnrate=optimal['learn_rate']
epochs=optimal['epochs']


# train the model with hyperparameters determined above
model.train(x_train,y_train,epochs=epochs,learn_rate=learnrate, batch_size=batchsize, 
              display_step=1, save_files=True, file_name=modelname, 
              save_model=True,model_name=modelname, random_state=1)

# *note there is a function model.fit(x_train,y_train, ... ) that wraps 
# get_hyperparameters() and train(), so that you do not have to manually
# pass hyperparameters to train(). However, get_hyperparameters() can
# be expensive, so I recommend running it once per dataset and using those
# hyperparameter values in train() to save computational time

Epoch: 0354 cost= 0.021673120 accuracy= 0.972
Epoch: 0355 cost= 0.087214798 accuracy= 0.972
Epoch: 0356 cost= 0.068042113 accuracy= 0.981
Epoch: 0357 cost= 0.050614341 accuracy= 0.972
Epoch: 0358 cost= 0.082408695 accuracy= 0.981
Epoch: 0359 cost= 0.052890776 accuracy= 0.972
Epoch: 0360 cost= 0.033155525 accuracy= 0.972
Epoch: 0361 cost= 0.078106005 accuracy= 0.972
Epoch: 0362 cost= 0.024788824 accuracy= 0.972
Epoch: 0363 cost= 0.051075329 accuracy= 0.981
Epoch: 0364 cost= 0.088189038 accuracy= 0.972
Epoch: 0365 cost= 0.155427599 accuracy= 0.981
Epoch: 0366 cost= 0.043871756 accuracy= 0.981
Epoch: 0367 cost= 0.035968403 accuracy= 0.981
Epoch: 0368 cost= 0.064268299 accuracy= 0.981
Epoch: 0369 cost= 0.046780230 accuracy= 0.972
Epoch: 0370 cost= 0.029981864 accuracy= 0.972
Epoch: 0371 cost= 0.040634620 accuracy= 0.981
Epoch: 0372 cost= 0.064101240 accuracy= 0.981
Epoch: 0373 cost= 0.030364695 accuracy= 0.972
Epoch: 0374 cost= 0.112073299 accuracy= 0.972
Epoch: 0375 cost= 0.085720089 accu

In [ ]:
# make predictions
m=model.predict(x_test) #returns the median prediction if more than one tree

#evaluate results
acc=sklearn.metrics.accuracy_score(y_test,m.flatten())  
print('Accuracy',acc)

#close model 
model.close_model()

INFO:tensorflow:Restoring parameters from ./class_djinn_test_tree0.ckpt
Model 0 restored
(1, 30, 3)
Accuracy 1.0


In [ ]:
print("Reload model and continue training")
# reload model; can also open it using cPickle.load()
model2=djinn.load(model_name="class_djinn_test")

#continue training for 20 epochs using same learning rate, etc as before
model2.continue_training(x_train, y_train, 20, learnrate, batchsize, random_state=1)

#make updated predictions
m2=model2.predict(x_test)

#evaluate results
acc=sklearn.metrics.accuracy_score(y_test,m.flatten())  
print('Accuracy',acc)

Reload model and continue training
INFO:tensorflow:Restoring parameters from ./class_djinn_test_tree0.ckpt
Model 0 restored
INFO:tensorflow:Restoring parameters from ./class_djinn_test_tree0.ckpt
Model 0 restored
Epoch: 0001 cost= 0.061807822 accuracy= 0.975
Epoch: 0002 cost= 0.038533498 accuracy= 0.983
Epoch: 0003 cost= 0.041990371 accuracy= 0.975
Epoch: 0004 cost= 0.076095770 accuracy= 0.983
Epoch: 0005 cost= 0.019222159 accuracy= 0.983
Epoch: 0006 cost= 0.062706901 accuracy= 0.975
Epoch: 0007 cost= 0.021604018 accuracy= 0.983
Epoch: 0008 cost= 0.050991039 accuracy= 0.975
Epoch: 0009 cost= 0.013415539 accuracy= 0.975
Epoch: 0010 cost= 0.049798382 accuracy= 0.983
Epoch: 0011 cost= 0.048999392 accuracy= 0.983
Epoch: 0012 cost= 0.046476536 accuracy= 0.983
Epoch: 0013 cost= 0.045946145 accuracy= 0.983
Epoch: 0014 cost= 0.062279145 accuracy= 0.983
Epoch: 0015 cost= 0.035428625 accuracy= 0.983
Epoch: 0016 cost= 0.049651360 accuracy= 0.975
Epoch: 0017 cost= 0.030312766 accuracy= 0.975
Epoch

In [ ]:
# Bayesian formulation with dropout. Recommend dropout keep 
# probability ~0.95, 5-10 trees.
print("Bayesian djinn example")
ntrees=3
dropout_keep=0.95
modelname="class_bdjinn_test"

# initialize a model
bmodel=djinn.DJINN_Classifier(ntrees,maxdepth,dropout_keep)

# "fit()" does what get_hyperparameters + train does, in one step: 
bmodel.fit(x_train,y_train, display_step=1, save_files=True, file_name=modelname, 
           save_model=True,model_name=modelname, random_state=1)

# evaluate: niters is the number of times you evaluate the network for 
# a single sample. higher niters = better resolved distribution of predictions
niters=100
bl,bm,bu,results=bmodel.bayesian_predict(x_test,n_iters=niters, random_state=1)
# bayesian_predict returns 25, 50, 75 percentile and results dict with all predictions

# evaluate performance on median predictions
acc=sklearn.metrics.accuracy_score(y_test,bm.flatten())  
print('Accuracy',acc)

# collect_tree_predictions gathers predictions in results dict
# in a more intuitive way for easy plotting, etc
p=bmodel.collect_tree_predictions(results['predictions'])

Epoch: 0754 cost= 0.027195086 accuracy= 0.991
Epoch: 0755 cost= 0.008973958 accuracy= 0.981
Epoch: 0756 cost= 0.025906423 accuracy= 0.991
Epoch: 0757 cost= 0.008528909 accuracy= 0.981
Epoch: 0758 cost= 0.043845693 accuracy= 0.991
Epoch: 0759 cost= 0.113019795 accuracy= 0.981
Epoch: 0760 cost= 0.013954947 accuracy= 0.981
Epoch: 0761 cost= 0.083741372 accuracy= 0.981
Epoch: 0762 cost= 0.056632050 accuracy= 0.981
Epoch: 0763 cost= 0.071708186 accuracy= 0.963
Epoch: 0764 cost= 0.080410318 accuracy= 0.981
Epoch: 0765 cost= 0.009985022 accuracy= 0.991
Epoch: 0766 cost= 0.026495805 accuracy= 0.991
Epoch: 0767 cost= 0.042551864 accuracy= 0.991
Epoch: 0768 cost= 0.008932173 accuracy= 0.991
Epoch: 0769 cost= 0.031113374 accuracy= 0.981
Epoch: 0770 cost= 0.085687706 accuracy= 0.991
Epoch: 0771 cost= 0.109489521 accuracy= 0.972
Epoch: 0772 cost= 0.082543499 accuracy= 0.981
Epoch: 0773 cost= 0.022839768 accuracy= 0.972
Epoch: 0774 cost= 0.097354406 accuracy= 0.991
Epoch: 0775 cost= 0.048759181 accu

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=de072003-a9db-4342-8067-19a4b45feff1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>